In [4]:
import praw
import pandas as pd
import datetime
import re

In [5]:
from dotenv import load_dotenv
load_dotenv()

ModuleNotFoundError: No module named 'dotenv'

In [11]:
import os
# REDDIT_CLIENT_ID = os.getenv('REDDIT_CLIENT_ID')
# REDDIT_CLIENT_SECRET = os.getenv('REDDIT_CLIENT_SECRET')
# REDDIT_USER_AGENT = os.getenv('REDDIT_USER_AGENT')

REDDIT_CLIENT_ID = 'zYdDXB3Ntg3zWzGSZVwNCQ'
REDDIT_CLIENT_SECRET = 'JNU1Wx-n9Xy9tZfviPsb_gSA0lB-Gg'
REDDIT_USER_AGENT = 'script:reddit_stock_scraper:v1.0 (by /u/AmoebaOld7828)'

In [10]:
reddit = praw.Reddit(
    client_id='zYdDXB3Ntg3zWzGSZVwNCQ',
    client_secret='JNU1Wx-n9Xy9tZfviPsb_gSA0lB-Gg',
    user_agent='script:reddit_stock_scraper:v1.0 (by /u/AmoebaOld7828)'
)

try:
    print("Authenticated as:", reddit.user.me())
except Exception as e:
    print("Failed to connect:", e)

Authenticated as: None


In [7]:
subreddits = ['investing', 'wallstreetbets', 'stocks']
keywords = ['NVDA', 'AAPL', 'GOOGL', 'AMZN', 'MSFT']

for subreddit_name in subreddits:
    subreddit = reddit.subreddit(subreddit_name)
    for post in subreddit.new(limit=100):
        title = post.title.upper()
        if any(keyword in title for keyword in keywords):
            print(f"{subreddit_name}: {post.title} (Score: {post.score}, Comments: {post.num_comments})")

investing: Should I move some of my NVDA stock to S&P500 or something else? (Score: 7, Comments: 30)
investing: AMD, ADI, MPWR or NXPI? Who is going to profit after Nvda? (Score: 1, Comments: 28)
wallstreetbets: Screw TSLA, should have gone with NVDA in the first place (Score: 58, Comments: 19)
wallstreetbets: Thank You NVDA and META ! Time to diversify and slow down for the summer. What you all regards think ? (Score: 43, Comments: 20)
stocks: Is NVIDIA Getting Too Hot? My Short-Term Concerns with NVDA ($141.92) (Score: 13, Comments: 6)
stocks: Thinking about picking up some Google, Apple, Broadcom and NVIDIA stock. Thoughts? (Score: 31, Comments: 112)


In [14]:
import praw
import re
import datetime
import csv
import os

In [15]:
# # CONFIGURATION
# REDDIT_CLIENT_ID = os.getenv('REDDIT_CLIENT_ID')         # or replace with your actual client ID as a string
# REDDIT_CLIENT_SECRET = os.getenv('REDDIT_CLIENT_SECRET') # or replace with your actual client secret
# REDDIT_USER_AGENT = os.getenv('REDDIT_USER_AGENT')       # or replace with your actual user agent
SUBREDDITS = ['investing', 'wallstreetbets', 'stocks']
TICKERS = ['GOOGL', 'AAPL', 'MSFT', 'NVDA', 'AMZN']
POST_LIMIT = 1000  # per subreddit
OUTPUT_FILE = 'reddit_posts.csv'
# -----------------------------------

reddit = praw.Reddit(client_id=REDDIT_CLIENT_ID,
                     client_secret=REDDIT_CLIENT_SECRET,
                     user_agent=REDDIT_USER_AGENT)

def extract_tickers(text):
    found = []
    for ticker in TICKERS:
        if re.search(r'\b' + re.escape(ticker) + r'\b', text.upper()):
            found.append(ticker)
    return found

all_posts = []

for sub_name in SUBREDDITS:
    print(f"Scraping r/{sub_name}...")
    subreddit = reddit.subreddit(sub_name)
    for post in subreddit.new(limit=POST_LIMIT):
        combined_text = (post.title + ' ' + (post.selftext or '')).upper()
        tickers_found = extract_tickers(combined_text)
        if not tickers_found:
            continue  # Skip if no relevant ticker

        all_posts.append({
            'title': post.title,
            'selftext': post.selftext,
            'author': str(post.author),
            'created_utc': datetime.datetime.utcfromtimestamp(post.created_utc),
            'score': post.score,
            'subreddit': sub_name,
            'tickers_mentioned': ','.join(tickers_found)
        })

# Write to CSV
with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['title', 'selftext', 'author', 'created_utc', 'score', 'subreddit', 'tickers_mentioned']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for post in all_posts:
        writer.writerow(post)

Scraping r/investing...


/var/folders/vr/nc9jvld12q115x_g_114k8g40000gn/T/ipykernel_65231/1181834452.py:37: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  'created_utc': datetime.datetime.utcfromtimestamp(post.created_utc),


Scraping r/wallstreetbets...
Scraping r/stocks...


In [16]:
# Convert to DataFrame
df = pd.DataFrame(all_posts)
df.to_csv(OUTPUT_FILE, index=False)
print(f"\n Saved {len(df)} posts to {OUTPUT_FILE}")


 Saved 161 posts to reddit_posts.csv


In [17]:
data = pd.read_csv("reddit_posts.csv")

In [19]:
data.head()

,title,selftext,author,created_utc,score,subreddit,tickers_mentioned
0,Should I move some of my NVDA stock to S&P500 ...,"Currently, I have ~$27,000 of NVDA stock. I di...",tortoise2022,2025-06-05 17:34:06,10,investing,NVDA
1,Costco Vs NVidia compare P/E and growth rates,"Costco trades at higher PE than NVDA, with low...",Dizzy_Maybe8225,2025-06-05 11:18:02,0,investing,NVDA
2,"AMD, ADI, MPWR or NXPI? Who is going to profit...",Looking at these chip makers trying to compare...,danrennt98,2025-06-04 17:09:53,0,investing,NVDA
3,"Sold Hood and started positions in Apple, Goog...",Recently I sold all my Hood shares (I got 4300...,coopermug,2025-05-30 01:27:26,0,investing,NVDA
4,Semis not recovering despite new wave of AI sp...,Looking at how SMH did recently I can't help t...,SurveyIllustrious738,2025-05-29 07:45:20,6,investing,NVDA


In [21]:
import pandas as pd
import numpy as np
import re

data['selftext'] = data['selftext'].fillna("")
data['created_utc'] = pd.to_datetime(data['created_utc'], errors='coerce')
data = data[data['created_utc'].notnull()]
data['full_text'] = data['title'] + " " + data['selftext']
def clean_text(text):
    text = text.lower()  # lowercase
    text = re.sub(r'http\S+', '', text)  # remove URLs
    text = re.sub(r'[^a-zA-Z0-9\s$]', '', text)  # remove special characters except $
    text = re.sub(r'\s+', ' ', text).strip()  # normalize whitespace
    return text

data['clean_text'] = data['full_text'].apply(clean_text)
data['author'] = data['author'].str.strip().str.lower()
data['subreddit'] = data['subreddit'].str.lower().str.strip()
data['tickers_mentioned'] = data['tickers_mentioned'].apply(
    lambda x: [t.strip().upper() for t in x.split(',')] if isinstance(x, str) else []
)

data.rename(columns={'created_utc': 'date'}, inplace=True)
data.drop_duplicates(subset=['clean_text', 'date', 'author'], inplace=True)
data.sort_values('date', inplace=True)

In [22]:
data.to_csv('../../data/clean-data/reddit-posts.csv')